In [1]:
# %load qbo_fub2netcdf.py
#!/usr/bin/python3
'''
Read QBO data from the following file: http://www.geo.fu-berlin.de/met/ag/strat/produkte/qbo/qbo.dat
'''

import matplotlib as mpl
from matplotlib import dates
import datetime
import xarray as xr
import pandas as pd
import numpy as np


def readsing2(nmonth,nyear):
    headerlines = []
    ye = []
    dats = []
    count = 0
    date = []
    #url = "http://www.geo.fu-berlin.de/met/ag/strat/produkte/qbo/singapore.dat"
    #download(url,"Data/singapore.dat")
    with open('../QBO/qbo_data/singapore.dat') as file:
        for i in range(3):
            headerlines.append(file.readline().strip())
        for year in range(1987,2018):
            if year == nyear:
                data = np.zeros([nmonth,15])*np.nan
            else:
                data = np.zeros([12,15])*np.nan
            ye.append(file.readline().strip())
            file.readline()
            if (year < 1997):
                data = np.zeros([12,15])*np.nan
                for i in range(14):
                    cols = file.readline()
                    cols = cols.strip().split()
                    for j in range(1,13):
                        if i == 1:
                            date.append(datetime.datetime(year, j, 1))
                        data[j-1,i] = float(cols[j])
            else:
                for i in range(15):
                    cols = file.readline()
                    cols = cols.strip().split()
                    for j in range(1,13):
                        if j < nmonth+1 or year < nyear:
                            if i == 1:
                                # print(year)
                                date.append(datetime.datetime(year, j, 1))
                            data[j-1,i] = float(cols[j])
                        # data[j-1,i] = float(cols[j])
            dats.extend([list(i) for i in data])
            file.readline()
            count = count + 1
#    print(np.shape(dats))
#    print(np.shape(np.array(dats)))
    pressure = [100, 90, 80, 70, 60, 50, 45, 40, 35, 30, 25, 20, 15, 12, 10]
    altitude = -7*np.log(np.array(pressure)/1013.15)
    fds = list(mpl.dates.date2num(date))
#    for i in range(np.shape(dats)[0]-np.shape(fds)[0]):
#        fds.append(np.nan)
    fds = np.array(fds)
    # print(np.shape(fds))
    return np.array(dats).T[::-1],fds,pressure,altitude

def readsing():
    #url = "http://www.geo.fu-berlin.de/met/ag/strat/produkte/qbo/qbo.dat"
    #download(url,"Data/singapore_winds_1953-2017_qbo.dat")
    data = np.genfromtxt('../QBO/qbo_data/qbo.dat',skip_header=9,dtype=['S6','S4','i4','i1','i4','i1','i4','i1','i4','i1','i4','i1','i4','i1','i4','i1'],names=['station','date','p70','n70','p50','n50','p40','n40','p30','n30','p20','n20','p15','n15','p10','n10'],delimiter=[6,4,6,2,5,2,5,2,5,2,5,2,5,2,5,2],filling_values=-999999,missing_values=' ')#
    date = []
    for i in range(len(data)):
        if int(data['date'][i]) > 5000:
            date.append(datetime.datetime.strptime('19'+(data['date'][i]).decode('UTF-8'),'%Y%m'))
        else:
            date.append(datetime.datetime.strptime('20'+(data['date'][i]).decode('UTF-8'),'%Y%m'))

    # print(date[-1])
    fds = mpl.dates.date2num(date)
#
    up = np.array(list([data['p70'],data['p50'],data['p40'],data['p30'],data['p20'],data['p15'],data['p10']]))

    pressure = [70,50,40,30,20,15,10]
    altitude = -7*np.log(np.array(pressure)/1013.15)
#    print(up)
    return up,fds,pressure,altitude

def convert2netcdf(time,u,pressure,fname):
    # Assuming the time variable is in pandas datetime format
    #time_datetime = pd.to_datetime(time, unit='D', origin='julian')
    # Convert Matplotlib date numbers to pandas datetime
    #time_datetime = pd.to_datetime(time, unit='D', origin=mpl.dates.date2num('0001-01-01'))
    time_datetime = time
    # Create xarray dataset
    ds = xr.Dataset({
        'u': (['pressure', 'time'], u),  # Calculate zonal mean
                    }, coords={
                                'pressure': pressure,
                                'time': time_datetime,
                                })

    # Add metadata
    ds['pressure'].attrs = {'standard_name': 'air_pressure', 'units': 'hPa'}
    ds['u'].attrs = {'standard_name': 'eastward_wind', 'units': 'm/s', 'cell_methods': 'time: mean'}

    # Global attributes
    ds.attrs = {'Conventions': 'CF-1.8', 'title': 'Zonal Mean Zonal Winds'}

    # Write to NetCDF file
    ds.to_netcdf(fname)

#def main():
    # -------------READ----------------------------------------------
up1,fds1,pressure1,altitude1 = readsing()
tnmoth = np.shape(up1)[1] # total number of months
up = np.zeros([15,tnmoth])*np.nan
nmonth = np.shape(up1)[1]%12 # number of months above a whole year
nyear = mpl.dates.num2date(fds1[-1]).year
up2,fds2,pressure,altitude = readsing2(nmonth,nyear)
fds = fds1
fds[-fds2.size:] = fds2
fds = np.array(fds)

up1 = up1*1.
up1[up1 < -10000] = np.nan
up[3,:] = up1[0,:]
up[5,:] = up1[1,:]
up[7,:] = up1[2,:]
up[9,:] = up1[3,:]
up[11,:] = up1[4,]
up[12,:] = up1[5,:]
up[14,:] = up1[6,:]
up[-up2.shape[0]:,-up2.shape[1]:] = up2
ind = ~np.isnan(up)
time,press = np.meshgrid(fds,pressure)
uu = np.array(up[ind])*.1
#    time = time[ind]
#    press = press[ind]    
fname = 'qbo_singapore_fub.nc'
time = mpl.dates.num2date(fds)
date = pd.to_datetime(time)
convert2netcdf(date,up*0.1,pressure,fname)
    

In [94]:
time

[datetime.datetime(1953, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 2, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 3, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 4, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 5, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 6, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 7, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 8, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 9, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 10, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1953, 12, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1954, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1954, 2, 1, 0, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(1954, 3, 1, 0, 0, tzinfo=d

In [48]:
print(15*766)

11490


In [59]:
print(np.size(up.reshape(766,15)))

11490


In [71]:
np.shape(up.T)

(766, 15)

In [57]:
import numpy as np

# Create a one-dimensional array
one_dim_array = np.array([1, 2, 3, 4, 5, 6])

# Reshape the one-dimensional array into a two-dimensional array
# The -1 in the reshape function means that the size of that dimension is inferred
two_dim_array = one_dim_array.reshape(-1, 2)

print("Original one-dimensional array:")
print(one_dim_array)

print("\nReshaped two-dimensional array:")
print(two_dim_array)

Original one-dimensional array:
[1 2 3 4 5 6]

Reshaped two-dimensional array:
[[1 2]
 [3 4]
 [5 6]]


In [80]:
pressure1

[70, 50, 40, 30, 20, 15, 10]

In [82]:
pressure

[100, 90, 80, 70, 60, 50, 45, 40, 35, 30, 25, 20, 15, 12, 10]

In [95]:
ds = xr.open_dataset('qbo_singapore_fub.nc')

In [96]:
ds

<xarray.Dataset>
Dimensions:   (pressure: 15, time: 849)
Coordinates:
  * pressure  (pressure) int64 100 90 80 70 60 50 45 40 35 30 25 20 15 12 10
  * time      (time) int64 -536457600000000000 ... 1512086400000000000
Data variables:
    u         (pressure, time) float64 ...
Attributes:
    Conventions:  CF-1.8
    title:        Zonal Mean Zonal Winds